In [ ]:
!pip install itertools

In [1]:
import pandas as pd
import numpy as np
import pytz
import time
import itertools
from datetime import datetime, timedelta
from datetime import datetime, timezone
from mt5orders import Execution
from smatrader import Indicators
from modi import Modify
import MetaTrader5 as mt5

In [2]:
#If MT5 isn't logged in, you have to log in manually or input your login details
initilized = mt5.initialize()

if initilized:
    print('Connected to MetaTrader5')
    print('Login:', mt5.account_info().login)
    print('Server:', mt5.account_info().server)

#Input the following parameters for trading
symbols_to_trade = ['XAUUSD']
lot_size = 0.1
stoploss_pips = 50
takeprofit_pips = 120   #Trailing stoploss, I highly recommend setting a 50% pips gain

#Miscellaneous stuff
past_data = 300         #Number of days you want to get the data
deviation = 20          #Deviation of the bid and ask price
resample_time = "1min"  #Candlestick resampling
specific_timezone = 2   #GMT + specific_timezone

#===================================================================================================================#
#This is the loop which is required to keep on running to capture, collect and execute trades
#It references to 3 other jupyter notebooks therefore all the files within the documents are equally important
#Do not touch anything unless you're sure of what you are doing
initial = Indicators(symbols_to_trade, mt5.TIMEFRAME_M5, past_data, resample_time, specific_timezone)
strategy = Execution(symbols_to_trade, lot_size, stoploss_pips, takeprofit_pips, deviation)
modifier = Modify(symbols_to_trade, lot_size, stoploss_pips, takeprofit_pips, deviation)

#Fetching data requires a few seconds for the python script to load
#Always make sure algo trading is off during the initialisation process
#And no on-going trades otherwise bot will not run

count = 0

while True:
    for sym in symbols_to_trade:
        if mt5.positions_total() == 0:
            initial.fetch_and_process_data()
            signal = initial.get_signal(sym)
            # print(f"Signal for {sym}: {signal}")

            if signal == 'neutral':
                count += 1
                print(count, end=' ')
                if signal == 'buy':
                    strategy.open_order_buy()
                elif signal == 'sell':
                    strategy.open_order_sell()

        elif mt5.positions_total() != 0:
            profits = modifier.calculate_profit()
            initial.fetch_and_process_data()

            signal = initial.get_signal(sym)
            print(f"Signal for {sym}: {signal}")
            print(profits)
            time.sleep(5)

            if signal == 'neutral':
                positions = mt5.positions_get()
                pos1 = positions[0]
                modifier.closing_order(pos1)

        else:
            pass
    time.sleep(2)

Connected to MetaTrader5
Login: 24046914
Server: FivePercentOnline-Real
1 2 3 4 

KeyboardInterrupt: 